In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['idf-mat-file', 'quora-insincere-questions-classification']


In [2]:
import os
import time
import tensorflow as tf
import numpy as np # linear algebra
import random
import os 
os.environ['PYTHONHASHSEED'] = '11'
np.random.seed(22)
random.seed(33)
tf.set_random_seed(44)

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import gc

from keras import initializers, regularizers, constraints
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPooling1D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from keras.callbacks import Callback
from keras.models import clone_model
import keras.backend as K

Using TensorFlow backend.


In [3]:
max_features = 90000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50
t0 = time.time()

In [4]:
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
##test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
##print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)


Extratcing the stopwords in English language

In [5]:
from nltk import corpus
stopwords=corpus.stopwords.words('english')

# fill up the missing values
train_X = train_df["question_text"].fillna("_na_").values
#test_X = test_df["question_text"].fillna("_na_").values


Stemming (result includes words not in stopword list)

In [6]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

train_X = train_df['question_text']
train_X = train_X.tolist()

qid_train = train_df['qid']
qid_train = qid_train.tolist()

##test_X = test_df['question_text']
##test_X = test_X.tolist()


stemming_result_train = {}
ps = PorterStemmer()
for i in range(0,len(qid_train)):
    words = word_tokenize(train_X[i])
    stemming_result_train[qid_train[i]] = []
    for w in words:
        if w not in stopwords:
            stemming_result_train[qid_train[i]].append(ps.stem(w))
            

##stemming_result_test = []
##for i in range(len(test_X)):
    ##for w in test_X[i]:
        ##stemming_result_test.append(ps.stem(w))
    
# stemming_result_test

Other Preprocesing : converting into lower case, extracting only alphabetical characters

In [7]:
import re

stemmed_content_train = {}
lowered_content_train = {}
alphabet_content_train = {}
for qid, words in stemming_result_train.items():
    stemmed_content_train[qid] = " ".join(str(word) for word in words)
    lowered_content_train[qid] = stemmed_content_train[qid].lower()
    alphabet_content_train[qid] = re.sub(r'[^a-zA-Z ]',"", lowered_content_train[qid])
      
    
##stemmed_content_test = []
##lowered_content_test = []
##alphabet_content_test = []
##for i in range(len(stemming_result_test)):
    ##stemmed_content_test.append(" ".join(str(word) for word in stemming_result_test[i]))
    ##lowered_content_test.append(stemmed_content_test[i].lower())
    #alphabet_content[qid] = lowered_content[qid].replace('[^a-z ]','')
    ##alphabet_content_test.append(re.sub(r'[^a-zA-Z ]',"", lowered_content_test[i]))
    
preprocessed_content1_train = []
##preprocessed_content1_test = []

for qid, text in alphabet_content_train.items():
    preprocessed_content1_train.append(alphabet_content_train[qid])

##for qid, text in alphabet_content_test.items():
##   preprocessed_content1_test.append(alphabet_content_test[qid])

Other preprocessing : Tokenization, Padding/Truncating questions

In [8]:
# ## split to train and val
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features,
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'’“”')


tokenizer.fit_on_texts(preprocessed_content1_train)
train_X = tokenizer.texts_to_sequences(preprocessed_content1_train)

# Pad the sentences 
trunc = 'pre'
train_X = pad_sequences(train_X, maxlen=maxlen, truncating=trunc)

##test_X = pad_sequences(test_X, maxlen=maxlen, truncating=trunc)

# Get the target values
train_y = train_df['target'].values

Splitting data into Train and Test

In [9]:
test_X = train_X[1000000:]
train_X = train_X[:1000000]
test_y = train_y[1000000:]
train_y = train_y[:1000000]

Creating Glove Embedding

In [10]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Wiki News Embedding

In [11]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
        
del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Paragram Embedding

In [12]:
EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector

del embeddings_index; gc.collect() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


0

Creating Gensim Embedding

In [13]:
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
from gensim.models import KeyedVectors

EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_4 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix_4[i] = embedding_vector
        
del embeddings_index; gc.collect()

/opt/conda/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


0

In [14]:
del EMBEDDING_FILE, stemmed_content_train,lowered_content_train,alphabet_content_train
del preprocessed_content1_train
del qid_train, stemming_result_train
del stopwords
del train_df

In [15]:
gc.collect()

7

Calculating IDF

In [16]:
from scipy.io import loadmat
idf = loadmat('../input/idf-mat-file/IDF.mat')

In [ ]:
word_extra_features=idf['word_extra_features']
vocab_word_freq=idf['vocab']
embed_size = 1
emb_mean, emb_std = word_extra_features.mean(), word_extra_features.std()
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_5 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    if word in vocab_word_freq:
        embedding_vector = word_extra_features[list(vocab_word_freq).index(word)]
        embedding_matrix_5[i] = embedding_vector

Concatinating the embeddings

In [ ]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4, embedding_matrix_5), axis=1)

del idf, embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4, embedding_matrix_5
gc.collect()
np.shape(embedding_matrix)

Utilities for Neural Network model

In [ ]:
class ExponentialMovingAverage(Callback):
    def __init__(self, model, decay=0.999, mode='epoch', n=100):
        """
        mode: 'epoch': Do update_weights every epoch.
              'batch':                   every n batches.
        n   :
        """
        self.decay = decay
        self.mode = mode
        self.ema_model = clone_model(model)
        self.ema_model.set_weights(model.get_weights())
        self.n = n
        if self.mode is 'batch':
            self.cnt = 0
        self.ema_weights = [K.get_value(w) for w in model.trainable_weights]
        self.n_weights = len(self.ema_weights)
        super(ExponentialMovingAverage, self).__init__()

    def on_batch_end(self, batch, logs={}):
        if self.mode is 'batch':
            self.cnt += 1
            if self.cnt % self.n == 0:
                self.update_weights()

    def on_epoch_end(self, epoch, logs={}):
        if self.mode is 'epoch':
            self.update_weights()
        for var, w in zip(self.ema_model.trainable_weights, self.ema_weights):
            K.set_value(var, w)

    def update_weights(self):
        for w_old, var_new in zip(self.ema_weights, self.model.trainable_weights):
            w_old += (1 - self.decay) * (K.get_value(var_new) - w_old)

Attention Function

In [ ]:
from keras.engine.topology import Layer

class Attention(Layer):
    def __init__(self, step_dim=50,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [ ]:
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization

def create_rnn_model(rnn, lstm, maxlen, embedding, max_features, embed_size,
                     rnn_dim=64, dense1_dim=100, dense2_dim=50,
                     embed_trainable=False, seed=123):
    inp = Input(shape=(maxlen,), dtype='int32')
    x = Embedding(max_features, embed_size, weights=[embedding],
                  trainable=embed_trainable)(inp)
    #x = Bidirectional(rnn(rnn_dim, return_sequences=True,
                          #kernel_initializer=glorot_uniform(seed=seed)))(x)
    x = Bidirectional(lstm(rnn_dim, kernel_initializer=glorot_uniform(seed=seed),
                           return_sequences=True))(x)
    
    
    x = Attention(maxlen)(x)
    #x = GlobalMaxPooling1D()(x)
    merged = Dense(dense1_dim, activation='relu')(x)
    merged = BatchNormalization()(merged)
    preds = Dense(1, activation='sigmoid')(merged)
    model = Model(inputs=[inp], \
        outputs=preds)
    model.compile(loss='binary_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])
    return model
    
    #x = Attention(maxlen)(x)
    #x = Bidirectional(lstm(rnn_dim, kernel_initializer=glorot_uniform(seed=seed),
                           #return_sequences=True))(x)
    #x = AttentionDecoder(rnn_dim,
                             #name='attention_decoder_1',
                             #output_dim=1,
                             #return_probabilities=False,
                             #trainable=True)(x) 
    #x =  Flatten()
    ##x = Dense(1, activation='sigmoid')
    #x = GlobalMaxPooling1D()(x)
    #x = Dense(dense2_dim, activation='relu',
              #kernel_initializer=glorot_uniform(seed=seed))(x)
    #x = Dense(1, activation='sigmoid',
              #kernel_initializer=glorot_uniform(seed=seed))(x)
    #model = Model(inputs=inp, outputs=x)
    #model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
    #return model

In [ ]:
k = 8
num_val_samples = 200000 
num_epochs = 10
all_scores = []

In [ ]:
def f1_best(y_val, pred_val):
    best_f1 = 0
    best_thresh = 0
    for thresh in np.linspace(0.2, 0.4, 41):
        f1 = metrics.f1_score(y_val, (pred_val > thresh).astype(int))
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    return best_f1, best_thresh

embed_ids = [list(range(300)), list(range(300, 600)),
             list(range(600, 900)), list(range(900, 1200))]
embed_ids_dict = {1: [embed_ids[0], embed_ids[1], embed_ids[2], embed_ids[3]],
                  2: [embed_ids[0] + embed_ids[1],
                      embed_ids[0] + embed_ids[2],
                      embed_ids[0] + embed_ids[3],
                      embed_ids[1] + embed_ids[2],
                      embed_ids[1] + embed_ids[3],
                      embed_ids[2] + embed_ids[3]],
                  3: [embed_ids[0] + embed_ids[1] + embed_ids[2],
                      embed_ids[0] + embed_ids[1] + embed_ids[3],
                      embed_ids[0] + embed_ids[2] + embed_ids[3],
                      embed_ids[1] + embed_ids[2] + embed_ids[3]],
                  4: [embed_ids[0] + embed_ids[1] + embed_ids[2] + embed_ids[3]]}
embed_ids_lst = embed_ids_dict[2]
embed_size = 600

rnn = CuDNNGRU
lstm = CuDNNLSTM
embed_trainable = False

n_models = 6
#epochs = 7
batch_size = 512
dense1_dim = rnn_dim = 128
dense2_dim = 2 * rnn_dim

ema_n = int(len(train_y) / batch_size / 10)
decay = 0.9
scores = []

#pred_avg = np.zeros((len(val_y), 1))
#pred_test_avg = np.zeros((test_df.shape[0], 1))
for i in range(n_models):
    t1 = time.time()
    seed = 101 + 11 * i
    cols_in_use = embed_ids_lst[i % len(embed_ids_lst)]
    model = create_rnn_model(rnn, lstm, maxlen, embedding_matrix[:, cols_in_use],
                             max_features, embed_size,
                             rnn_dim=rnn_dim,
                             dense1_dim=dense1_dim,
                             dense2_dim=dense2_dim,
                             embed_trainable=embed_trainable,
                             seed=seed)
    model.summary()
    ema = ExponentialMovingAverage(model, decay=decay, mode='batch', n=ema_n)
    
    #cross_validation 
    for i in range(k):
        print(f'Processing fold # {i}')
        val_data = train_X[i * num_val_samples: (i+1) * num_val_samples]
        val_targets = train_y[i * num_val_samples: (i+1) * num_val_samples]
    
        partial_train_data = np.concatenate(
                                [train_X[:i * num_val_samples],
                                train_X[(i+1) * num_val_samples:]],
                                axis=0)
        partial_train_targets = np.concatenate(
                                [train_y[:i * num_val_samples],
                                train_y[(i+1)*num_val_samples:]],
                                axis=0)
        res = model.fit(x=partial_train_data,
                        y=partial_train_targets,
                        batch_size=batch_size,
                        epochs = num_epochs,
                        verbose=1,
                        validation_data = (val_data,val_targets))
        print("Scores: ", res)
    
    #model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs,
              #callbacks=[ema], verbose=0)
    m = ema.ema_model
    t_per_epoch = (time.time() - t1) / epochs
    pred = m.predict([val_X])
    pred_avg += pred
    pred_test = m.predict([test_X])
    pred_test_avg += pred_test
    f1_one, thresh_one = f1_best(val_y, pred)
    f1_avg, thresh_avg = f1_best(val_y, pred_avg / (i + 1))
    nll_one = metrics.log_loss(val_y, pred)
    nll_avg = metrics.log_loss(val_y, pred_avg / (i + 1))
    auc_one = metrics.roc_auc_score(val_y, pred)
    auc_avg = metrics.roc_auc_score(val_y, pred_avg)
    print(f'  n_model:{i + 1} epoch:{epochs} F1:{f1_avg:.4f} th:{thresh_avg:.3f} ' +
          f'AUC:{auc_avg:.4f} NLL:{nll_avg:.4f} One:{f1_one:.4f} {auc_one:.4f} {nll_one:.4f} ' +
          f'Time:{time.time() - t1:.1f}s  {t_per_epoch:.1f}s/epoch')

In [ ]:
pred_test_avg /= n_models
pred_test_avg = (pred_test_avg>thresh_avg).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_avg
out_df.to_csv("submission.csv", index=False)

print(f'Done:{time.time() - t0:.1f}s')

In [ ]:
print(out_df)